# Introduction to Pandas

This tutorial is partially based on the ["Solving real world data science tasks with Python Pandas!"](https://github.com/KeithGalli/Pandas-Data-Science-Tasks) tutorial, by Keith Galli.

The [Sales dataset](https://github.com/KeithGalli/Pandas-Data-Science-Tasks/tree/master/SalesAnalysis/Sales_Data) used in this tutorial comes from the aforementioned Keith's tutorial.

## Import necessary libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

## Load and explore sales data for one month 

### Load sales data from a csv file

The data is stored in the 'sales_data' subdirectory, within the 'data' directory. 

In [2]:
DATA_DIR = "data/sales_data"

Load the data from the 'Sales_January_2019' csv file, from the 'sales_data' directory 

In [3]:
jan_sales = pd.read_csv(Path.cwd() / DATA_DIR / 'Sales_January_2019.csv')

### Explore the loaded 

Let's first see what the data looks like

In [4]:
jan_sales.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


First, get the size of the dataframe, that is, the number of rows and columns

In [5]:
jan_sales.shape

(9723, 6)

Access the columns' names

In [6]:
jan_sales.columns.to_list()

['Order ID',
 'Product',
 'Quantity Ordered',
 'Price Each',
 'Order Date',
 'Purchase Address']

Examine the columns' data types

In [7]:
jan_sales.dtypes

Order ID            object
Product             object
Quantity Ordered    object
Price Each          object
Order Date          object
Purchase Address    object
dtype: object

A nice overview of pandas' data types can be found, for example, in [this article](https://pbpython.com/pandas_dtypes.html).

Get a complete set of information about the data frame

In [8]:
jan_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9723 entries, 0 to 9722
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Order ID          9697 non-null   object
 1   Product           9697 non-null   object
 2   Quantity Ordered  9697 non-null   object
 3   Price Each        9697 non-null   object
 4   Order Date        9697 non-null   object
 5   Purchase Address  9697 non-null   object
dtypes: object(6)
memory usage: 455.9+ KB


### Examine the missing values

To do that, we'll use the `isna()` method

In [9]:
jan_sales.isna().sum()

Order ID            26
Product             26
Quantity Ordered    26
Price Each          26
Order Date          26
Purchase Address    26
dtype: int64

#### Remove rows with all missing values

This can be done with the `dropna()` method

In [10]:
jan_sales.dropna(how='all', inplace=True)

In [11]:
jan_sales.isna().sum().sum()

0

### Subset the data frame or select particular rows / columns

This can be done in different ways; the recommended practice is to use Dataframe methods `loc` or `iloc`.

When doing the selection based on the ordinal number of rows and columns, `iloc` should be used.

When the selection is to be based on a condition to be evaluated against rows and / or columns are to be selected based on their names, `loc` should be used. 

**Example 1**: select first 3 columns and first 6 rows

In [12]:
jan_sales.iloc[:6, :3]

,Order ID,Product,Quantity Ordered
0,141234,iPhone,1
1,141235,Lightning Charging Cable,1
2,141236,Wired Headphones,2
3,141237,27in FHD Monitor,1
4,141238,Wired Headphones,1
5,141239,AAA Batteries (4-pack),1


**Example 2**: select the last 5 rows and 1st, 2nd and 5th column

In [13]:
jan_sales.iloc[-5:, [0,1,4]]

,Order ID,Product,Order Date
9718,150497,20in Monitor,01/26/19 19:09
9719,150498,27in FHD Monitor,01/10/19 22:58
9720,150499,ThinkPad Laptop,01/21/19 14:31
9721,150500,AAA Batteries (4-pack),01/15/19 14:21
9722,150501,Google Phone,01/13/19 16:43


**Example 3**: Select Order ID, Order Date, and Purchase Address for orders coming from Boston

Note: we can use `str` to get the string value of a pandas Series (the 'Purchase Address' column, in this case), so that we can call any string method on it.

In [14]:
jan_sales.loc[jan_sales['Purchase Address'].str.contains('Boston'), ['Order ID', 'Order Date', 'Purchase Address']]

,Order ID,Order Date,Purchase Address
0,141234,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
17,141251,01/24/19 08:13,"414 Walnut St, Boston, MA 02215"
29,141263,01/11/19 23:33,"640 Spruce St, Boston, MA 02215"
31,141265,01/01/19 16:52,"853 Ridge St, Boston, MA 02215"
37,141271,01/30/19 10:51,"90 13th St, Boston, MA 02215"
...,...,...,...
9627,150409,01/10/19 02:38,"410 Forest St, Boston, MA 02215"
9660,150440,01/10/19 19:19,"82 Johnson St, Boston, MA 02215"
9666,150445,01/11/19 13:56,"147 Lincoln St, Boston, MA 02215"
9694,150473,01/10/19 09:09,"148 Hickory St, Boston, MA 02215"


**Example 4**: Select Order ID, Product and Price Each for orders where the ordered quantity is at least 3

Note that even though the Quantity Ordered column stores numeric information, its values are encoded as strings: 

In [15]:
type(jan_sales['Quantity Ordered'][0])

str

So, we first need to transform the Quantity Ordered column to numeric type. <br>To that end, we use `pd.to_numeric()` function:

In [16]:
jan_sales['Quantity Ordered'] = pd.to_numeric(jan_sales['Quantity Ordered'], errors='coerce')

Now, we can do the intended selection:

In [17]:
jan_sales.loc[jan_sales['Quantity Ordered'] >= 3, ['Order ID', 'Price Each', 'Product']]

,Order ID,Price Each,Product
12,141246,2.99,AAA Batteries (4-pack)
52,141285,2.99,AAA Batteries (4-pack)
58,141290,3.84,AA Batteries (4-pack)
73,141305,2.99,AAA Batteries (4-pack)
84,141316,2.99,AAA Batteries (4-pack)
...,...,...,...
9513,150299,3.84,AA Batteries (4-pack)
9590,150372,2.99,AAA Batteries (4-pack)
9636,150417,11.99,Wired Headphones
9659,150439,11.99,Wired Headphones


## Load sales data for all months in 2019 and merge them into one data set

We need to load the sales data from all csv files (one for each month) available in the 'sales_data' directory. 

Let's first see how to take only .csv files from a given directory:

In [18]:
def get_csv_files(fpath):
    csv_files = []
    if not fpath.is_dir():
        print("Error! Not a directory! Cannot proceed")
        return
    for f in fpath.iterdir():
        if f.is_dir(): continue
        if f.suffix == ".csv":
            csv_files.append(f)
    return csv_files

In [19]:
get_csv_files(Path.cwd() / DATA_DIR)

[PosixPath('/Users/jelena.jovanovic/DataspellProjects/v2023/pandas/data/sales_data/Sales_December_2019.csv'),
 PosixPath('/Users/jelena.jovanovic/DataspellProjects/v2023/pandas/data/sales_data/Sales_April_2019.csv'),
 PosixPath('/Users/jelena.jovanovic/DataspellProjects/v2023/pandas/data/sales_data/Sales_February_2019.csv'),
 PosixPath('/Users/jelena.jovanovic/DataspellProjects/v2023/pandas/data/sales_data/Sales_March_2019.csv'),
 PosixPath('/Users/jelena.jovanovic/DataspellProjects/v2023/pandas/data/sales_data/Sales_August_2019.csv'),
 PosixPath('/Users/jelena.jovanovic/DataspellProjects/v2023/pandas/data/sales_data/Sales_May_2019.csv'),
 PosixPath('/Users/jelena.jovanovic/DataspellProjects/v2023/pandas/data/sales_data/Sales_November_2019.csv'),
 PosixPath('/Users/jelena.jovanovic/DataspellProjects/v2023/pandas/data/sales_data/Sales_October_2019.csv'),
 PosixPath('/Users/jelena.jovanovic/DataspellProjects/v2023/pandas/data/sales_data/Sales_January_2019.csv'),
 PosixPath('/Users/jelena

We'll now take each csv file in turn, load the data from it and add the loaded data into a common DataFrame that stores sales data for the whole year.

To merge two DataFrames, we'll use `pd.concat` function

In [20]:
pd.concat?

In [21]:
all_sales = pd.DataFrame()

for sales_file in get_csv_files(Path.cwd() / DATA_DIR):
    sales_data = pd.read_csv(sales_file)
    all_sales = pd.concat([all_sales, sales_data])

all_sales.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"


### Examine the merged data

In [22]:
all_sales.shape

(186850, 6)

In [23]:
all_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186850 entries, 0 to 13621
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 10.0+ MB


Note the difference between the number of entries (rows) and the index values. It seems that some of the loaded files have overlaping indexes. To fix this, we need to reset the index of the merged data frame.

In [24]:
all_sales.reset_index(drop=True, inplace=True)

In [25]:
all_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


Rename the DataFrame, to reduce the typing: 

In [26]:
sales = all_sales

Note that we are just giving a new name (reference) to the same object, that is, we are not creating a new DataFrame object with this statement. <br>
We can verify that by comparing the two data frame references:

In [27]:
sales is all_sales

True

To create a new DataFrame object, we would need to do, for example, the following:

In [28]:
tmp = all_sales.copy()
tmp is all_sales

False

In [29]:
del tmp

In [30]:
del all_sales

## Prepare the sales data for analysis 

Let's take another look at the data...

In [31]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


Note that all the data are stored as object (string) values. We will need to change that for the Quantity Ordered, Price Each and Order Date columns. But before that, we will handle missing values.

### Remove observations with all missing values

It seems that entire rows of data are missing, that is, we have observations with all missing values. <br>Let's take a sample to check that:

In [32]:
sales.loc[sales.Product.isna(),].sample(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
66357,NaN,NaN,NaN,NaN,NaN,NaN
10489,NaN,NaN,NaN,NaN,NaN,NaN
48129,NaN,NaN,NaN,NaN,NaN,NaN
75440,NaN,NaN,NaN,NaN,NaN,NaN
124107,NaN,NaN,NaN,NaN,NaN,NaN
124943,NaN,NaN,NaN,NaN,NaN,NaN
86918,NaN,NaN,NaN,NaN,NaN,NaN
26550,NaN,NaN,NaN,NaN,NaN,NaN
75050,NaN,NaN,NaN,NaN,NaN,NaN
67472,NaN,NaN,NaN,NaN,NaN,NaN


Remove observations with all missing values

In [33]:
sales.dropna(how='all', inplace=True)

In [34]:
sales.isna().sum().sum()

0

### Convert Quantity Ordered and Price Each to numeric values, and Order Date into datetime type

A useful reference for this type of conversion can be found in [this StackOverflow thread](https://stackoverflow.com/questions/15891038/change-column-type-in-pandas)

Let's start by taking another look at the columns' data types

In [35]:
sales.dtypes

Order ID            object
Product             object
Quantity Ordered    object
Price Each          object
Order Date          object
Purchase Address    object
dtype: object

First, we'll try to convert Quantity Ordered to a numeric column

In [36]:
try:
    sales['Quantity_Ordered'] = pd.to_numeric(sales['Quantity Ordered'], errors='raise')
except ValueError as err:
    print(err)

Unable to parse string "Quantity Ordered" at position 254


Take a look at the entry (observation) where the error occurred

In [37]:
sales.iloc[254, :]

Order ID                    Order ID
Product                      Product
Quantity Ordered    Quantity Ordered
Price Each                Price Each
Order Date                Order Date
Purchase Address    Purchase Address
Name: 254, dtype: object

This looks like some erroneous data that we do not need. So, we will do the conversion (to numeric type) and, along the way, replace such erronous data with NaN (that is, mark the data as missing)  

In [38]:
sales['Quantity_Ordered'] = pd.to_numeric(sales['Quantity Ordered'], errors='coerce')

Get the number and proportion of the introduced missing values in the Quantity Ordered column

In [39]:
sales.Quantity_Ordered.isna().sum()

355

In [40]:
sales.Quantity_Ordered.isna().mean()

0.0019054775770913288

Compute the basic statistics for the newly created numeric column

In [41]:
sales.Quantity_Ordered.describe()

count    185950.000000
mean          1.124383
std           0.442793
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           9.000000
Name: Quantity_Ordered, dtype: float64

In [42]:
sales.Quantity_Ordered.value_counts()

Quantity_Ordered
1.0    168552
2.0     13324
3.0      2920
4.0       806
5.0       236
6.0        80
7.0        24
8.0         5
9.0         3
Name: count, dtype: int64

Do the same for the Price Each column

In [43]:
sales['Price_Each'] = pd.to_numeric(sales['Price Each'], errors='coerce')

In [44]:
sales.Price_Each.describe()

count    185950.000000
mean        184.399735
std         332.731330
min           2.990000
25%          11.950000
50%          14.950000
75%         150.000000
max        1700.000000
Name: Price_Each, dtype: float64

In [45]:
sales.Price_Each.isna().sum()

355

Drop rows where ordered quantity or unit price are unavailable (NaN)

In [46]:
rows_to_drop = sales.loc[sales.Quantity_Ordered.isna() | sales.Price_Each.isna(),].index

In [47]:
sales.drop(rows_to_drop, inplace=True)

Check the size of the dataset after the removal of rows with missing values

In [48]:
sales.shape

(185950, 8)

Finally, transform the Order Date to datetime type

In [49]:
sales['Order_Date'] =  pd.to_datetime(sales['Order Date'], format="%m/%d/%y %H:%M", errors='coerce')

In [50]:
sales.Order_Date.isna().sum()

0

In [51]:
sales[['Order_Date', 'Order Date']].sample(5)

,Order_Date,Order Date
131256,2019-10-01 20:52:00,10/01/19 20:52
17487,2019-12-18 17:31:00,12/18/19 17:31
138993,2019-01-31 23:10:00,01/31/19 23:10
125863,2019-10-04 14:07:00,10/04/19 14:07
151817,2019-09-03 10:52:00,09/03/19 10:52


It seems that this transformation went well. 

Let's check the data types after the transformations done above

In [52]:
sales.dtypes

Order ID                    object
Product                     object
Quantity Ordered            object
Price Each                  object
Order Date                  object
Purchase Address            object
Quantity_Ordered           float64
Price_Each                 float64
Order_Date          datetime64[ns]
dtype: object

Remove columns that are no longer needed

In [53]:
sales.drop(columns=['Quantity Ordered', 'Price Each', 'Order Date'], inplace=True)
sales.columns.to_list()

['Order ID',
 'Product',
 'Purchase Address',
 'Quantity_Ordered',
 'Price_Each',
 'Order_Date']

### Add Purchase City column

Let's see how we can create a new column out of an existing one.

Since we may be interested in comparing sales across cities, we will add a column with the name of the city where the order was made. The city name should be followed by the state label, given in brackets (e.g. 'Atlanta (GA)'), to disambigute cities with common names.   

Take a sample of values of the Purchase Address column, to examine how addresses are formatted

In [54]:
sales['Purchase Address'].sample(10)

8210          453 8th St, San Francisco, CA 94016
29097      678 Cherry St, San Francisco, CA 94016
161045        927 7th St, San Francisco, CA 94016
17219     414 Jackson St, San Francisco, CA 94016
56952        439 Park St, San Francisco, CA 94016
23592       974 Johnson St, Los Angeles, CA 90001
181588          910 Madison St, Atlanta, GA 30301
62388              857 Pine St, Atlanta, GA 30301
173338          763 2nd St, Los Angeles, CA 90001
52034          64 8th St, San Francisco, CA 94016
Name: Purchase Address, dtype: object

We will start by creating a function that can extract the city (and state label) from the Purchase Address column. 

In [55]:
def get_city(address):
    street, city, state = address.split(',')
    st_label, st_zip = state.lstrip().split()
    return f"{city.lstrip()} ({st_label})"

Next, we will *apply* the `get_city` function to each value of the Purchase Address series. First, on a sample of addresses:

In [56]:
sales['Purchase Address'].sample(10).apply(get_city)

11473     New York City (NY)
182305           Austin (TX)
178980    San Francisco (CA)
115741           Boston (MA)
145111    San Francisco (CA)
111251    New York City (NY)
107721    San Francisco (CA)
79477           Atlanta (GA)
29400          Portland (ME)
89116            Boston (MA)
Name: Purchase Address, dtype: object

Then, on all `Purchase Address` values, to create the new column `Purchase City`

In [57]:
sales['Purchase_City'] = sales['Purchase Address'].apply(get_city)

In [58]:
sales[['Purchase Address', 'Purchase_City']].sample(10)

,Purchase Address,Purchase_City
77307,"106 Main St, Boston, MA 02215",Boston (MA)
105410,"242 Jackson St, Los Angeles, CA 90001",Los Angeles (CA)
76384,"970 Lakeview St, Austin, TX 73301",Austin (TX)
155850,"619 Spruce St, Atlanta, GA 30301",Atlanta (GA)
179292,"245 6th St, Atlanta, GA 30301",Atlanta (GA)
2298,"768 Maple St, New York City, NY 10001",New York City (NY)
57774,"875 Chestnut St, Austin, TX 73301",Austin (TX)
81188,"865 Dogwood St, San Francisco, CA 94016",San Francisco (CA)
143197,"242 14th St, Atlanta, GA 30301",Atlanta (GA)
16010,"858 11th St, Atlanta, GA 30301",Atlanta (GA)


Save the transformed dataset to a file so that we do not have to go through the data cleaning and transformation step again

In [59]:
sales.to_csv(Path.cwd() / DATA_DIR / 'all_sales_2019.csv', index=False)

## Exploratory data analysis

Let's now try to answer some sales-related questions by exploring the 2019 sales data

### Task 1. What was the best month for sales? How much was earned that month?

To answer this question, we need to:
- compute total revenue per order
- extract month from the Order Date, so that we can group orders on the montly basis
- compute total revenue per month

**Step 1**: Compute total revenue per order

**Step 2**: Extract month from the Order Date

Note that Order Date is a datetime Series. We can use `dt` to get access to each value in the Series as a datetime object. <br>Having access to a particular order date as a datetime object, we can take any element of the order date and time through appropriate fields (year, month, day, etc.)

**Step 3**: Compute total revenue per month

Sort the results in the decreasing order of monthly total revenue

We can see that the last month, December, had the highest revenue and that it was ~4.6M USD.

We can also examine monthly revenues on a (bar) chart, as visual representation of data often facilitates the comprehension of trends that may be present in the data.

For plotting, we will use the Matplotlib library. Its documentation is available [here](https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html), while [this example](https://matplotlib.org/stable/gallery/lines_bars_and_markers/bar_colors.html#sphx-glr-gallery-lines-bars-and-markers-bar-colors-py) can be helpful as the starting point for this task.

Note: [This GitHub repo](https://github.com/matplotlib/cheatsheets) offers very well done cheat sheets and tips for Mathplotlib

### Task 2. What city ordered the largest quantity of products?

This time we will group orders based on the city. <br>
Recall that we have created Purchase City column out of the Purchase Address column

Sort the results in the decreasing order of the per-city ordered quantity

Finally, plot the per-city ordered quantites on a bar chart

### Task 3. When, during a day, customers are likely to make most valuable orders?

To answer this one, we can extract the hour from the Order Date and examine the average total order price across the 24 hours of a day

**Step 1**: Create the 'Order Hour' column

**Step 2**: Group orders per Order Hour and compute mean Total price for each group

Sort the results in the descending order of the mean total price

Examine the results on a line chart

### Task 4. What product sold the most? What product contributed the most to the total revenue?

Let's first see how many different products there are and how frequently they were ordered

While the above series gives us the number of orders that a product appeared in, it does not tell us about the quantities that were ordered.

For each product, we need to take the overall quantity ordered and total price across all orders related to that product

Get the products (top 5) that sold the most:

Get the products (top 5) with the highest total revenue

We can also plot these data. <br>
In particular, we will create two horizontal bar plots, one next to the other, the first one showing total revenue per product, the second depicting the quantity of the product sold. We will sort the plots based on the total revenue, so that the products with the highest revenue are displayed towards the top of the plots.

For the guidance on how to create a horizontal bar chart, see [this example](https://matplotlib.org/stable/gallery/lines_bars_and_markers/barh.html) from the Mathplotlib documentation.

We can also plot this data on a single line plot, using different scales for values shown on the y-axis. 

[This example](https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html#sphx-glr-gallery-subplots-axes-and-figures-two-scales-py) from the MatPlotLib documentation shows how have a plot with two different scales.

### Task 5. Examine total revenue per product across cities

Note that we can group the data based on more than one attribute (column). <br> In this case, we will do the grouping based on the product and the city and compute total revenue for each product in each city:

Note that we are getting a Series with two indexes - Product and Purchase City:

We can transform this into a DataFrame by calling the `unstack()` method

We can now plot this DataFrame using a heathmap. 
A handy guide for creating a heatmap is given in [this article](https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html#sphx-glr-gallery-images-contours-and-fields-image-annotated-heatmap-py) from the Matplotlib documentation.

See [this page](https://matplotlib.org/stable/users/explain/colors/colormaps.html) of Matplotlib documentation for an overview of the predefined color maps.

### Tasks to practice

1. Examine the distribution of total product purchases, in terms of total revenue, across the states. <br><br>
2. On which day, during a week, customers are likely to make most valuable orders?